In [ ]:
import pandas as pd
import datetime
import numpy as np
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def ticker_selection(x):

    tickers = []
    i = 0
    data = pd.DataFrame()
    tickers.append(input("Please continue with ticker selection, press \"end\" to finish  "))

    while tickers[i] != "end":
        i += 1
        tickers.append(input("Please continue with ticker selection, type \"end\" to finish  "))
        
    else:
        print("Security Selection Ended")
        del tickers[i]
    
    start_date = input("Select Start Date yy-mm-dd ")
    end_date = input("Select End Date yy-mm-dd / Type \"current\" for current ")
    
    if end_date == "current":
        end_date = str(datetime.datetime.today())
    
    
    for i in tickers:
        
        data[i] = wb.DataReader(i, data_source = 'yahoo', start = start_date, end = end_date)['Adj Close']
    
    return data
    


In [ ]:
sec_data = ticker_selection(1)
    
portfolio_returns = []
portfolio_std = []
weights_list = []
for i in range(1000):
    weights = np.random.random(len(sec_data.columns))
    weights = (weights / np.sum(weights))      
    weights = np.round(weights,1)
    portfolio_returns.append(np.dot(weights,np.log(sec_data/sec_data.shift(1)).mean() * 250))
    portfolio_std.append(np.sqrt(np.dot(weights.T,np.dot(np.log(sec_data/sec_data.shift(1)).cov() * 250,weights))))
    weights_list.append(weights)

    
portfolio_returns = np.array(portfolio_returns)
portfolio_std = np.array(portfolio_std)
weights_list = pd.DataFrame(weights_list)
weights_list.columns = sec_data.columns


portfolios = pd.DataFrame({"Returns" : portfolio_returns, "Volatility" : portfolio_std})
portfolios = pd.concat([portfolios,weights_list], axis = 1)    
portfolios
   

In [ ]:
plot = portfolios.plot(x = "Volatility", y = "Returns", kind = "scatter", figsize = (15,20))
plt.ylabel("Expected Returns")
plt.xlabel("Expected Volatility")
    
plt.show()   

In [ ]:
 
df1 = portfolios[portfolios["Returns"] == portfolios["Returns"].max()]
df2 = portfolios[portfolios["Returns"] == portfolios["Returns"].min()]
df3 = portfolios[portfolios["Volatility"] == portfolios["Volatility"].max()]
df4 = portfolios[portfolios["Volatility"] == portfolios["Volatility"].min()]
df5 = portfolios[(portfolios['Returns']/portfolios['Volatility']) == (portfolios['Returns']/portfolios['Volatility']).max ()]

df = pd.concat([pd.DataFrame(df1.iloc[0]).transpose(), pd.DataFrame(df2.iloc[0]).transpose(),pd.DataFrame(df3.iloc[0]).transpose(),pd.DataFrame(df4.iloc[0]).transpose(),pd.DataFrame(df5.iloc[0]).transpose()])

extreme_portfolios = df.rename(index = {df.index[0] : "Maximum Return Portfolio", df.index[1] : "Minimum Return Portfolio" , df.index[2] : "Maximum Volatility Portfolio", df.index[3] : "Minimum Volatility Portfolio", df.index[4] : "Maximum Sharpe" })

extreme_portfolios
